# PART A

In [1]:
# imports
import os
import time
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import wandb
from env import create_standard_grid, create_four_room

# Optional: configure matplotlib for notebooks
%matplotlib inline


### Create environment

In [2]:
all_configs = []

# -------------------------------
# 10x10 Grid World: Q-learning (8)
# -------------------------------
for tp in [0.7, 1.0]:
    for start_state in [(0, 4), (3, 6)]:
        for strat in ['eps_greedy', 'softmax']:
            name = f"std_q_tp{tp}_ss{start_state}_strat{strat}"
            cfg = {
                'name': name,
                'alg': 'q_learning',
                'env_builder': create_standard_grid,
                'env_kwargs': {
                    # ✅ FIX: convert tuple to 2D numpy array
                    'start_state': np.array([[start_state[0], start_state[1]]]),
                    'transition_prob': tp,
                    'wind': False
                },
                'strategy': strat
            }
            all_configs.append(cfg)

# -------------------------------
# 10x10 Grid World: SARSA (8)
# -------------------------------
for wind in [True, False]:
    for start_state in [(0, 4), (3, 6)]:
        for strat in ['eps_greedy', 'softmax']:
            name = f"std_sarsa_wind{wind}_ss{start_state}_strat{strat}"
            cfg = {
                'name': name,
                'alg': 'sarsa',
                'env_builder': create_standard_grid,
                'env_kwargs': {
                    # ✅ FIX here too
                    'start_state': np.array([[start_state[0], start_state[1]]]),
                    'transition_prob': 1.0,
                    'wind': wind
                },
                'strategy': strat
            }
            all_configs.append(cfg)

# -------------------------------
# Four-Room configs (no fix needed)
# -------------------------------
for goal_change in [True, False]:
    all_configs.append({
        'name': f"four_q_goalchange{goal_change}_strateps_greedy",
        'alg': 'q_learning',
        'env_builder': create_four_room,
        'env_kwargs': {'goal_change': goal_change},
        'strategy': 'eps_greedy'
    })
for goal_change in [True, False]:
    all_configs.append({
        'name': f"four_sarsa_goalchange{goal_change}_strateps_greedy",
        'alg': 'sarsa',
        'env_builder': create_four_room,
        'env_kwargs': {'goal_change': goal_change},
        'strategy': 'eps_greedy'
    })

### Functions

In [3]:
# Safe reward scalar extractor for 1-element arrays
def reward_scalar(r):
    return float(np.ravel(r)[0])

# epsilon greedy function
# Chose to use correct datatype outputs as it becomes a problem in downstream tasks.

def epsilon_greedy_action(Q: np.ndarray, state: int, epsilon: float) -> int:
    """Pick epsilon-greedy action from Q[state]."""
    if np.random.rand() < epsilon:
        return np.random.randint(Q.shape[1])
    return int(np.argmax(Q[state]))

# --- Episode termination check ---
def is_terminal(state: int, goal_states_seq: np.ndarray) -> bool:
    return state in set(map(int, np.array(goal_states_seq).flatten()))

# --- Rollouts for evaluation ---
def evaluate_policy(env, Q: np.ndarray, episodes: int = 20, max_steps: int = 100) -> float:
    total = 0.0
    for _ in range(episodes):
        s = env.reset()
        ep_ret = 0.0
        for _ in range(max_steps):
            a = int(np.argmax(Q[s]))
            s_next, r = env.step(s, a)
            ep_ret += reward_scalar(r)
            s = int(s_next)
            if is_terminal(s, env.goal_states_seq):
                break
        total += ep_ret
    return total / episodes
    
def softmax_action(Q, state, tau):
    tau = float(tau)
    if tau <= 0:
        return int(np.argmax(Q[state]))
    z = Q[state] / tau
    z = z - np.max(z)
    p = np.exp(z)
    p = p / np.sum(p)
    return int(np.random.choice(len(p), p=p))


def select_action(Q, state, strategy, param):
    if strategy == 'eps_greedy':
        if np.random.rand() < float(param):
            return int(np.random.randint(Q.shape[1]))
        return int(np.argmax(Q[state]))
    elif strategy == 'softmax':
        return softmax_action(Q, state, float(param))
    else:
        raise ValueError('Unknown exploration strategy: ' + str(strategy))

### Q-learning function

In [4]:
def train_q_learning_fixed(env, episodes, alpha, gamma, strategy, param, max_steps=100, run=None, seed=None):
    """
    Q-Learning with optional WandB logging.
    """
    n_states, n_actions = env.num_states, env.num_actions
    Q = np.zeros((n_states, n_actions))
    returns = []

    for ep in range(episodes):
        state = int(env.reset())
        total_reward = 0.0

        for _ in range(max_steps):
            a = select_action(Q, state, strategy, param)
            s_next, r = env.step(state, a)
            r = reward_scalar(r)
            total_reward += r
            s_next = int(s_next)

            if is_terminal(s_next, env.goal_states_seq):
                Q[state, a] += alpha * (r - Q[state, a])
                break

            td_target = r + gamma * np.max(Q[s_next])
            Q[state, a] += alpha * (td_target - Q[state, a])
            state = s_next

        returns.append(total_reward)

        # ✅ WandB logging
        if run is not None:
            wandb.log({
                "episode": ep,
                "episode_return": total_reward,
                "algorithm": "Q-learning",
                "alpha": alpha,
                "gamma": gamma,
                "param": param,
                "strategy": strategy,
                "seed": seed
            })

        if (ep + 1) % 200 == 0:
            print(f"[Q] Episode {ep+1}/{episodes} | Mean(Last50): {np.mean(returns[-50:]):.2f}")

    return Q, returns


### SARSA function

In [5]:
def train_sarsa_fixed(env, episodes, alpha, gamma, strategy, param, max_steps=100, run=None, seed=None):
    """
    SARSA with optional WandB logging.
    """
    n_states, n_actions = env.num_states, env.num_actions
    Q = np.zeros((n_states, n_actions))
    returns = []

    for ep in range(episodes):
        s = int(env.reset())
        a = select_action(Q, s, strategy, param)
        total_reward = 0.0

        for _ in range(max_steps):
            s_next, r = env.step(s, a)
            r = reward_scalar(r)
            total_reward += r
            s_next = int(s_next)

            if is_terminal(s_next, env.goal_states_seq):
                Q[s, a] += alpha * (r - Q[s, a])
                break

            a_next = select_action(Q, s_next, strategy, param)
            td_target = r + gamma * Q[s_next, a_next]
            Q[s, a] += alpha * (td_target - Q[s, a])
            s, a = s_next, a_next

        returns.append(total_reward)

        # ✅ WandB logging
        if run is not None:
            wandb.log({
                "episode": ep,
                "episode_return": total_reward,
                "algorithm": "SARSA",
                "alpha": alpha,
                "gamma": gamma,
                "param": param,
                "strategy": strategy,
                "seed": seed
            })

        if (ep + 1) % 200 == 0:
            print(f"[SARSA] Episode {ep+1}/{episodes} | Mean(Last50): {np.mean(returns[-50:]):.2f}")

    return Q, returns


### Setting up Wandb

In [6]:
'''wandb.init()

# 
sweep_config_1 = {
    'method': 'grid',   # try every combination
    'metric': {
        'name': 'mean_eval_return',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {'values': [0.001, 0.01, 0.1, 1.0]},
        'gamma': {'values': [0.7, 0.8, 0.9, 1.0]},
        'param': {'values': [0.001, 0.01, 0.05, 0.1]},  # Use param if your code expects epsilon as "param"
        'algorithm': {'values': ['q_learning']},
        'strategy': {'values': ['eps_greedy']},
        'env_name': {'values': ['std_q_tp0.7_ss(0,4)_strateps_greedy']}
    }
}'''

'wandb.init()\n\n# \nsweep_config_1 = {\n    \'method\': \'grid\',   # try every combination\n    \'metric\': {\n        \'name\': \'mean_eval_return\',\n        \'goal\': \'maximize\'\n    },\n    \'parameters\': {\n        \'alpha\': {\'values\': [0.001, 0.01, 0.1, 1.0]},\n        \'gamma\': {\'values\': [0.7, 0.8, 0.9, 1.0]},\n        \'param\': {\'values\': [0.001, 0.01, 0.05, 0.1]},  # Use param if your code expects epsilon as "param"\n        \'algorithm\': {\'values\': [\'q_learning\']},\n        \'strategy\': {\'values\': [\'eps_greedy\']},\n        \'env_name\': {\'values\': [\'std_q_tp0.7_ss(0,4)_strateps_greedy\']}\n    }\n}'

In [8]:
'''sweep_id = wandb.sweep(sweep_config_1, project="GPA_2_PART_A")
print("Sweep ID:", sweep_id)'''


'sweep_id = wandb.sweep(sweep_config_1, project="GPA_2_PART_A")\nprint("Sweep ID:", sweep_id)'

In [11]:
import numpy as np
import wandb
import random

def train_one_sweep_run():
    wandb.init()
    cfg = wandb.config

    n_seeds = 5  # number of seeds
    eval_scores = []
    final_mean_rewards = []

    for seed in range(n_seeds):
        np.random.seed(seed)
        random.seed(seed)

        # setup environment (make sure env creation uses np.random properly)
        env = create_standard_grid(transition_prob=0.7, start_state=np.array([[0, 4]]))

        # choose algorithm
        if cfg.algorithm == "q_learning":
            Q, returns = train_q_learning_fixed(
                env,
                episodes=1500,
                alpha=cfg.alpha,
                gamma=cfg.gamma,
                strategy=cfg.strategy,
                param=cfg.param,
                max_steps=100,
                run=None  # avoid spamming W&B with logs per seed
            )
        else:
            Q, returns = train_sarsa_fixed(
                env,
                episodes=1500,
                alpha=cfg.alpha,
                gamma=cfg.gamma,
                strategy=cfg.strategy,
                param=cfg.param,
                max_steps=100,
                run=None
            )

        # evaluate this seed
        avg_eval = evaluate_policy(env, Q, episodes=20, max_steps=100)

        eval_scores.append(avg_eval)
        final_mean_rewards.append(np.mean(returns[-50:]))

        wandb.log({
            'seed': seed,
            'seed_eval_return': avg_eval,
            'seed_final_mean_reward': np.mean(returns[-50:])
        })

    # aggregate over all seeds
    mean_eval = np.mean(eval_scores)
    std_eval = np.std(eval_scores)
    mean_final_reward = np.mean(final_mean_rewards)
    std_final_reward = np.std(final_mean_rewards)

    # log aggregated statistics
    wandb.log({
        'mean_eval_return': mean_eval,
        'std_eval_return': std_eval,
        'mean_final_mean_reward_last50': mean_final_reward,
        'std_final_mean_reward_last50': std_final_reward
    })

    print(
        f"Run finished | α={cfg.alpha}, γ={cfg.gamma}, param={cfg.param} "
        f"→ avg_eval={mean_eval:.2f} ± {std_eval:.2f}"
    )
    wandb.finish()



In [ ]:
'''def train_one_sweep_run_for_seeds():
    wandb.init()
    seeds = [1, 2, 3, 4, 5]
    eval_returns = []
    returns_list = []

    cfg = wandb.config

    for seed in seeds:
        set_seed(seed)
        # DO NOT call wandb.init() here!

        env = create_standard_grid(transition_prob=0.7, start_state=np.array([[0, 4]]))

        # ...train...

        avg_eval = evaluate_policy(env, Q, episodes=20, max_steps=100)
        eval_returns.append(avg_eval)
        returns_list.append(np.mean(returns[-50:]))

        wandb.log({
            'mean_eval_return': avg_eval,
            'final_mean_reward_last50': np.mean(returns[-50:]),
            'seed': seed
        })
        print(f"Seed {seed}... avg_eval={avg_eval:.2f}")

    # Log the mean across all seeds for the sweep run
    mean_eval = np.mean(eval_returns)
    mean_last_50 = np.mean(returns_list)
    wandb.log({
        'mean_eval_return_5seeds': mean_eval,
        'final_mean_reward_last50_5seeds': mean_last_50
    })
    wandb.finish()'''


In [ ]:
#wandb.agent(sweep_id, function=train_one_sweep_run, count=None)


In [12]:
sweep_config_2 = {
    'method': 'grid',   # try every combination
    'metric': {
        'name': 'mean_eval_return',
        'goal': 'maximize'
    },
    'parameters': {
        'alpha': {'values': [0.001, 0.01, 0.1, 1.0]},
        'gamma': {'values': [0.7, 0.8, 0.9, 1.0]},
        'param': {'values': [0.001, 0.01, 0.05, 0.1]},
        'algorithm': {'values': ['q_learning']},
        'strategy': {'values': ['eps_greedy']},
        'env_name': {'values': ['std_q_tp1.0_ss(0,4)_strateps_greedy']}
    }
}


In [13]:
sweep_id_2 = wandb.sweep(sweep_config_2, project="GPA_2_PART_A")
print("Sweep ID:", sweep_id_2)


Create sweep with ID: dn2upq25
Sweep URL: https://wandb.ai/leenh050418-iit-madras-foundation/GPA_2_PART_A/sweeps/dn2upq25
Sweep ID: dn2upq25


In [14]:
wandb.agent(sweep_id_2, function=train_one_sweep_run, count=None)

wandb: Agent Starting Run: ds8x8ucm with config:
wandb: 	algorithm: q_learning
wandb: 	alpha: 0.001
wandb: 	env_name: std_q_tp1.0_ss(0,4)_strateps_greedy
wandb: 	gamma: 0.7
wandb: 	param: 0.001
wandb: 	strategy: eps_greedy
wandb: Currently logged in as: leenh050418 (leenh050418-iit-madras-foundation) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


[Q] Episode 200/1500 | Mean(Last50): -117.18
[Q] Episode 400/1500 | Mean(Last50): -111.70
[Q] Episode 600/1500 | Mean(Last50): -107.54
[Q] Episode 800/1500 | Mean(Last50): -106.52
[Q] Episode 1000/1500 | Mean(Last50): -107.24
[Q] Episode 1200/1500 | Mean(Last50): -106.56
[Q] Episode 1400/1500 | Mean(Last50): -112.98
[Q] Episode 200/1500 | Mean(Last50): -111.56
[Q] Episode 400/1500 | Mean(Last50): -108.46
[Q] Episode 600/1500 | Mean(Last50): -110.16
[Q] Episode 800/1500 | Mean(Last50): -111.22
[Q] Episode 1000/1500 | Mean(Last50): -105.06
[Q] Episode 1200/1500 | Mean(Last50): -108.20
[Q] Episode 1400/1500 | Mean(Last50): -107.04
[Q] Episode 200/1500 | Mean(Last50): -113.84
[Q] Episode 400/1500 | Mean(Last50): -117.84
[Q] Episode 600/1500 | Mean(Last50): -112.90
[Q] Episode 800/1500 | Mean(Last50): -115.48
[Q] Episode 1000/1500 | Mean(Last50): -111.96
[Q] Episode 1200/1500 | Mean(Last50): -107.74
[Q] Episode 1400/1500 | Mean(Last50): -104.94
[Q] Episode 200/1500 | Mean(Last50): -114.36
[

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


Run finished | α=0.001, γ=0.7, param=0.001 → avg_eval=-133.22 ± 54.59


mean_eval_return,▁
mean_final_mean_reward_last50,▁
seed,▁▃▅▆█
seed_eval_return,█▇█▁█
seed_final_mean_reward,▂█▄▆▁
std_eval_return,▁
std_final_mean_reward_last50,▁
mean_eval_return,-133.22
mean_final_mean_reward_last50,-106.38
seed,4
seed_eval_return,-105.5


...

wandb: Agent Starting Run: cjblaeww with config:
wandb: 	algorithm: q_learning
wandb: 	alpha: 1
wandb: 	env_name: std_q_tp1.0_ss(0,4)_strateps_greedy
wandb: 	gamma: 1
wandb: 	param: 0.1
wandb: 	strategy: eps_greedy


[Q] Episode 200/1500 | Mean(Last50): -72.44
[Q] Episode 400/1500 | Mean(Last50): -43.62
[Q] Episode 600/1500 | Mean(Last50): -74.24
[Q] Episode 800/1500 | Mean(Last50): -51.80
[Q] Episode 1000/1500 | Mean(Last50): -42.86
[Q] Episode 1200/1500 | Mean(Last50): -61.88
[Q] Episode 1400/1500 | Mean(Last50): -36.72
[Q] Episode 200/1500 | Mean(Last50): -79.02
[Q] Episode 400/1500 | Mean(Last50): -63.30
[Q] Episode 600/1500 | Mean(Last50): -65.58
[Q] Episode 800/1500 | Mean(Last50): -38.78
[Q] Episode 1000/1500 | Mean(Last50): -49.84
[Q] Episode 1200/1500 | Mean(Last50): -62.94
[Q] Episode 1400/1500 | Mean(Last50): -37.28
[Q] Episode 200/1500 | Mean(Last50): -58.58
[Q] Episode 400/1500 | Mean(Last50): -65.52
[Q] Episode 600/1500 | Mean(Last50): -64.52
[Q] Episode 800/1500 | Mean(Last50): -95.24
[Q] Episode 1000/1500 | Mean(Last50): -80.46
[Q] Episode 1200/1500 | Mean(Last50): -75.98
[Q] Episode 1400/1500 | Mean(Last50): -91.42
[Q] Episode 200/1500 | Mean(Last50): -97.68
[Q] Episode 400/1500 | 

wandb: ERROR The nbformat package was not found. It is required to save notebook history.


mean_eval_return,▁
mean_final_mean_reward_last50,▁
seed,▁▃▅▆█
seed_eval_return,▂█▂▇▁
seed_final_mean_reward,▅▄▁█▆
std_eval_return,▁
std_final_mean_reward_last50,▁
mean_eval_return,-75.76
mean_final_mean_reward_last50,-75.12
seed,4
seed_eval_return,-118.25


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
